<a href="https://colab.research.google.com/github/MehrdadJannesar/CNN_AdaBoost/blob/master/ResNet50_laitec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import libraries
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense,Input,BatchNormalization
from keras.applications.resnet50 import ResNet50
from keras.datasets import mnist
import numpy as np

Using TensorFlow backend.


In [2]:
#Connect to google drive for pretrian weights
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#Load Dataset
#Optimize and categorize dataset
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()
X_train = train_images.astype('float32')
X_test = test_images.astype('float32')

X_train /= 255
X_test /= 255
N_classes = 10
Y_train = keras.utils.to_categorical(train_labels,N_classes)
Y_test = keras.utils.to_categorical(test_labels,N_classes)

170500096/170498071 [==============================] - 6s 0us/step


In [4]:
#Create model
"""
---> Input layer 
---> Default Resnet50 layers Keras
---> 10 classes fullyConnected 

I use the default ResNet layer in Keras, but in the next cell open all of the layers in summary.
"""
input_layer = Input(shape=(32,32,3))
resnet = ResNet50(include_top = False, pooling = 'avg'
                  , weights = 'imagenet')(input_layer)

dense = Dense(N_classes, activation = 'softmax')(resnet)

myModel = Model(input_layer, dense)














94658560/94653016 [==============================] - 3s 0us/step



In [5]:
#summary Model (Default ResNet in Keras)
myModel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [6]:
#Create new model from input_layer to last layer in ResNet for explaining summary
#The final output of this cell is showing each layer's detail in ResNet
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer(index=1).outputs
ress_model = Model(inputs=input_ress,outputs=output_ress)

ress_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [7]:
#Compile my model and Define optimizer
#Optimizer is SGD (Stochastic Gradient Descent) and loss function is Crossentropy
sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
myModel.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [10]:
#I create datetime for computing run time traning 

import datetime
#Start Time
start = datetime.datetime.now()
#Fit myModel
network = myModel.fit(X_train, Y_train, batch_size=128, epochs=10 )
#End Time
end = datetime.datetime.now()
#Computing Time
print('time compile is : ', str(end - start))

Epoch 1/10
50000/50000 [==============================] - 28s 552us/step - loss: 0.3289 - acc: 0.8840
Epoch 2/10
50000/50000 [==============================] - 27s 550us/step - loss: 0.2619 - acc: 0.9087
Epoch 3/10
50000/50000 [==============================] - 27s 547us/step - loss: 0.2581 - acc: 0.9106
Epoch 4/10
50000/50000 [==============================] - 27s 545us/step - loss: 0.2036 - acc: 0.9303
Epoch 5/10
50000/50000 [==============================] - 28s 551us/step - loss: 0.1522 - acc: 0.9479
Epoch 6/10
50000/50000 [==============================] - 27s 542us/step - loss: 0.1254 - acc: 0.9561
Epoch 7/10
50000/50000 [==============================] - 27s 544us/step - loss: 0.1188 - acc: 0.9589
Epoch 8/10
50000/50000 [==============================] - 27s 541us/step - loss: 0.0989 - acc: 0.9655
Epoch 9/10
50000/50000 [==============================] - 27s 540us/step - loss: 0.0850 - acc: 0.9723
Epoch 10/10
50000/50000 [==============================] - 27s 543us/step - loss: 

In [11]:
#Predict network
#Validation myModel (Loss,Accurcy)
test_loss, test_acc = myModel.evaluate(X_test, Y_test)
test_prediction = myModel.predict(X_test)


print("test_loss: ",test_loss)
print("test_acc: ",test_acc)
print("test_prediction", test_prediction)

10000/10000 [==============================] - 4s 449us/step
test_loss:  1.4872945759773255
test_acc:  0.7103
test_prediction [[7.8472135e-07 4.5912424e-05 4.6144766e-07 ... 8.6151166e-09
  3.4274501e-06 1.4716832e-05]
 [3.8903590e-02 3.6015627e-01 2.0619188e-04 ... 7.6950837e-06
  6.0048115e-01 8.7503569e-05]
 [9.9875784e-01 4.5345654e-04 2.4088546e-05 ... 3.2706629e-07
  7.5319369e-04 7.5057064e-06]
 ...
 [6.3695524e-08 1.9584673e-08 8.6513137e-06 ... 3.3608561e-03
  7.2201274e-08 8.9651202e-08]
 [3.2003712e-02 9.6642202e-01 7.4156451e-05 ... 1.4237674e-05
  6.0270058e-06 3.7203703e-05]
 [5.3919571e-06 2.8019347e-06 2.4959218e-04 ... 8.8276607e-01
  3.6082130e-07 6.2324659e-07]]


ResNet
Deep residual networks were a breakthrough idea which enabled the development of much deeper networks (hundreds of layers as opposed to tens of layers).

Its a generally accepted principle that deeper networks are capable of learning more complex functions and representations of the input which should lead to better performance. However, many researchers observed that adding more layers eventually had a negative effect on the final performance. This behavior was not intuitively expected, as explained by the authors below.

Let us consider a shallower architecture and its deeper counterpart that adds more layers onto it. There exists a solution by construction to the deeper model: the added layers are identity mapping, and the other layers are copied from the learned shallower model. The existence of this constructed solution indicates that a deeper model should produce no higher training error than its shallower counterpart. But experiments show that our current solvers on hand are unable to find solutions that are comparably good or better than the constructed solution (or unable to do so in feasible time).

This phenomenon is referred to by the authors as the degradation problem - alluding to the fact that although better parameter initialization techniques and batch normalization allow for deeper networks to converge, they often converge at a higher error rate than their shallower counterparts. In the limit, simply stacking more layers degrades the model's ultimate performance.

The authors propose a remedy to this degradation problem by introducing residual blocks in which intermediate layers of a block learn a residual function with reference to the block input. You can think of this residual function as a refinement step in which we learn how to adjust the input feature map for higher quality features. This compares with a "plain" network in which each layer is expected to learn new and distinct feature maps. In the event that no refinement is needed, the intermediate layers can learn to gradually adjust their weights toward zero such that the residual block represents an identity function.

![](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-6.29.19-PM.png)

Note: It was later discovered that a slight modification to the original proposed unit offers better performance by more efficiently allowing gradients to propagate through the network during training.

![](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-17-at-10.36.21-PM.png)

Wide residual networks
Although the original ResNet paper focused on creating a network architecture to enable deeper structures by alleviating the degradation problem, other researchers have since pointed out that increasing the network's width (channel depth) can be a more efficient way of expanding the overall capacity of the network.

Architecture

![](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-6.30.05-PM.png)

Each colored block of layers represent a series of convolutions of the same dimension. The feature mapping is periodically downsampled by strided convolution accompanied by an increase in channel depth to preserve the time complexity per layer. Dotted lines denote residual connections in which we project the input via a 1x1 convolution to match the dimensions of the new block.

The diagram above visualizes the ResNet 34 architecture. For the ResNet 50 model, we simply replace each two layer residual block with a three layer bottleneck block which uses 1x1 convolutions to reduce and subsequently restore the channel depth, allowing for a reduced computational load when calculating the 3x3 convolution.

![alt text](https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-5.47.38-PM.png)

Parameters: 25 million (ResNet 50)

Papers:

Deep Residual Learning for Image Recognition
Identity Mappings in Deep Residual Networks
Wide Residual Networks

Site Link:

[Site Link](https://www.jeremyjordan.me/convnet-architectures/)